# Possible causes for declining voter turnout in Germany

For decades, Germany had very high voter turnouts of up to 91% (1972) in its federal elections (Bundestagswahlen). But since 1987 and with some ups and downs, numbers have declined significantly to around 77% in the latest elections in 2021. In elections in federal states (Bundesländer) and on the local level (Kommunalwahlen) these numbers have been even far lower at times.

There is a lot of data available to search for possible causes of the decline in voter turnout. This project tries to find and visualize a small part of them.

## Data sources

The main data source is the "Bundeswahlleiter", the federal authority regarding elections in Germany. It publishes detailed election results by constituency (Wahlkreis) and also socio-economic data for all constituencies. Here are some links to the sources:

**Election results and voter turnout (Bundeswahlleiter)**
- Data for 2021 (including the election in 2017) ([HTML](https://www.bundeswahlleiter.de/bundestagswahlen/2021/ergebnisse/opendata.html#d5720c75-b642-497c-bea2-1c196f4a39c4), [CSV wide](https://www.bundeswahlleiter.de/bundestagswahlen/2021/ergebnisse/opendata/csv/kerg2.csv), [CSV long](https://www.bundeswahlleiter.de/bundestagswahlen/2021/ergebnisse/opendata/csv/kerg2.csv), [Dataset description](https://www.bundeswahlleiter.de/dam/jcr/f801a6d7-e51f-4804-baa4-dacec780704d/btw21_dsb_kerg2.pdf))
- Final results of all federal elections by constituency ([ZIP](https://www.bundeswahlleiter.de/dam/jcr/ce2d2b6a-f211-4355-8eea-355c98cd4e47/btw_kerg.zip))
- Results of past elections ([PDF](https://www.bundeswahlleiter.de/dam/jcr/397735e3-0585-46f6-a0b5-2c60c5b83de6/btw_ab49_gesamt.pdf))

**Socio-economic data for constituencies (Bundeswahlleiter):**
- Data for 2009 ([PDF](https://www.bundeswahlleiter.de/dam/jcr/4b83ac13-2c2c-43c8-bf78-ca6e50c15fb4/btw09_heft1.pdf))
- Data for 2013 ([HTML](https://www.bundeswahlleiter.de/bundestagswahlen/2013/strukturdaten.html), [CSV](https://www.bundeswahlleiter.de/dam/jcr/65ef1c2d-4df0-44e2-8881-99a176b4896c/btw2013_strukturdaten.csv))
- Data for 2017 ([HTML](https://www.bundeswahlleiter.de/bundestagswahlen/2017/strukturdaten.html), [CSV](https://www.bundeswahlleiter.de/dam/jcr/f7566722-a528-4b18-bea3-ea419371e300/btw2017_strukturdaten.csv))
- Data for 2021 ([HTML](https://www.bundeswahlleiter.de/bundestagswahlen/2021/strukturdaten.html), [CSV](https://www.bundeswahlleiter.de/dam/jcr/b1d3fc4f-17eb-455f-a01c-a0bf32135c5d/btw21_strukturdaten.csv))

**Other data regarding federal elections (Bundeswahlleiter):**
- Eligible voters, voters and turnout by gender and birth year groups ([HTML](https://www.bundeswahlleiter.de/bundestagswahlen/2021/ergebnisse/repraesentative-wahlstatistik.html), [CSV](https://www.bundeswahlleiter.de/dam/jcr/2aaec1fb-745a-422d-9ef0-0be7c9ca0ac9/btw21_rws_bw2.csv))<br>
<code>btw21_rws_bw2.csv</code>
- Time series since 1953: Eligible voters, voters and voter turnout by gender and age group ([HTML](https://www.bundeswahlleiter.de/bundestagswahlen/2021/ergebnisse/repraesentative-wahlstatistik.html), [CSV](https://www.bundeswahlleiter.de/dam/jcr/f920aa03-a0b1-45a5-8e65-7902a67259d7/btw_rws_wb-1953.csv))<br>
<code>btw_rws_wb-1953.csv</code>


## Import necessary libraries

In [ ]:
import json
import math
import os
import pathlib
from os.path import exists
from urllib.parse import urlparse

import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import requests
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression

## Define some settings, templates and functions for later use

In [ ]:
# Basic settings for the plots
conf = {
    "width": 1000,
    "height": 750,
}

# Define a custom template for the plots
custom_template = dict(
    layout=go.Layout(
        autosize=False,
        width=conf["width"],
        height=conf["height"],
        margin=dict(t=120, r=90, b=80, l=70),
        font=dict(
            family="Lato",
            size=14,
            color="#1f1f1f",
        ),
        title=dict(
            font=dict(
                family="Lato",
                size=32,
                color="#1f1f1f",
            ),
            x=0.02,
            y=0.95,
        ),
        xaxis=dict(
            automargin=True,
        ),
        yaxis=dict(
            automargin=True,
            showgrid=False,
        ),
        legend=dict(
            valign="top",
        ),
    )
)

In [ ]:
# Function to add an attribution the plots
def add_attribution(fig, data, x=-0.06, y=-0.14):

    source = f"<b>Data:</b> {data}, " if data else ""

    return fig.add_annotation(
        xref="paper",
        yref="paper",
        x=x,
        y=y,
        xanchor="left",
        yanchor="bottom",
        showarrow=False,
        text=f"{source}<b>Graph:</b> Jan Kühn (https://yotka.org), "
        "<b>License:</b> CC by-nc-sa 4.0",
    )


# Function to add a trendline to a plot
def add_trendline(fig, pos=0, row=False, col=False, line=None):

    x = fig["data"][pos]["x"]
    y = fig["data"][pos]["y"]

    if not line:
        line = dict(color="#333", width=0.5)

    # Calculate linear regression
    regr = LinearRegression()
    regr.fit(np.array(x).reshape(-1, 1), np.array(y))
    fit = regr.predict(np.array(x).reshape(-1, 1))

    # Create trace
    trace = go.Scatter(x=x, y=fit, mode="lines", line=line)

    # Add to subplots, if rows and cols are provided
    if row and col:
        return fig.add_trace(trace, row, col)

    # Add to single plot
    return fig.add_trace(trace)


# Function to set axis titles
def add_axis_titles(fig, title_x=None, title_y=None):

    # Set title for x-axis
    if title_x:

        # Remove default axis title
        fig.update_layout(
            xaxis=dict(title=""),
        )

        # Create new axis title
        fig.add_annotation(
            xref="paper",
            yref="paper",
            x=1,
            y=-0.07,
            xanchor="right",
            yanchor="bottom",
            showarrow=False,
            text=title_x,
            font=dict(size=14),
        )

    # Set title for y-axis
    if title_y:

        # Remove default axis title
        fig.update_layout(
            yaxis=dict(title=""),
        )

        # Create new axis title
        fig.add_annotation(
            xref="paper",
            yref="paper",
            x=-0.06,
            y=1,
            xanchor="left",
            yanchor="top",
            showarrow=False,
            text=title_y,
            textangle=270,
            font=dict(size=14),
        )

    return fig


# Add a custom title to the plot's colorbar
def add_colorbar_title(fig, title="Color & size: Difference to 2017 election"):

    # Add colorbar title
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=1.09,
        y=0.985,
        xanchor="right",
        yanchor="top",
        showarrow=False,
        text=title,
        textangle=90,
        font=dict(size=14),
    )

    return fig

## Get general election statistics

In [ ]:
# Import csv and have a look at it
stats = pd.read_csv(
    "data/grunddaten-btw-seit-1949.csv", sep=";", decimal=",", thousands="."
)
# stats

In [ ]:
# Visualize absolute numbers of eligible voters, voters, and non-voters
fig = px.line(
    stats,
    x="Jahr",
    y=["Wahlberechtigte", "Wähler", "Nichtwähler"],
    title="<b>Number of eligible voters grew much faster than voters</b><br />"
    "<sup>Absolute numbers of participation in German federal elections 1949-2021</sup>",
    template=custom_template,
)

fig.update_layout(
    margin=dict(r=30),
    xaxis=dict(
        tickmode="array",
        tickvals=list(stats["Jahr"]),
        automargin=True,
        range=[stats["Jahr"].min() - 5, stats["Jahr"].max() + 10],
        tickfont=dict(size=11),
    ),
    yaxis=dict(
        title="",
        range=[0, 70000000],
        automargin=True,
    ),
    showlegend=False,
)

fig.update_traces(
    line=dict(width=4),
)

translation = {
    "Wahlberechtigte": "Eligible voters",
    "Wähler": "Voters",
    "Nichtwähler": "Non-voters",
}

# Add label to last value of each line
for i, v in enumerate(fig.data):
    fig.add_scatter(
        x=[fig.data[i].x[-1] + 0.7],
        y=[fig.data[i].y[-1]],
        mode="text",
        text=[f"<b>{translation[fig.data[i]['name']]}"],
        textfont=dict(
            color=fig.data[i]["line"]["color"],
        ),
        textposition="middle right",
    )

# Add arrow
fig.add_annotation(
    x=1990,
    y=60430000,
    ax=-50,
    # ay=65000000,
    xanchor="right",
    yanchor="middle",
    text="First election<br />after reunification",
    font=dict(size=14),
    xshift=-5,
)

# Add axis titles
fig = add_axis_titles(fig, "Election year", "Absolute number")

# Add attribution
fig = add_attribution(fig, "Bundeswahlleiter")

fig.show()

In [ ]:
# Export as image
fig.write_image("export/01-absolute-voter-numbers.png", scale=1)

In [ ]:
# Visualize voter turnout
fig = px.line(
    stats,
    x="Jahr",
    y="Wahlbeteiligung",
    title="<b>Voter turnout in Germany declined since the 1980ies</b><br />"
    "<sup>Voter turnout in German federal elections since 1949</sup>",
    template=custom_template,
    text="Wahlbeteiligung",
)

fig.update_layout(
    margin=dict(r=30),
    xaxis=dict(
        tickmode="array",
        tickvals=list(stats["Jahr"]),
        range=[1946, 2024],
        tickfont=dict(size=11),
    ),
    yaxis=dict(
        rangemode="tozero",
        range=[50, 100],
    ),
)

fig.update_traces(
    texttemplate="%{text:.0f}<br />",
    textposition="top center",
    line=dict(width=4),
    mode="lines+text",
)

# Add axis titles
fig = add_axis_titles(fig, "Election year", "Voter turnout (%)")

# Add attribution
fig = add_attribution(fig, "Bundeswahlleiter")

fig.show()

In [ ]:
# Export as image
fig.write_image("export/02-voter-turnout.png", scale=1)

## Compare voter turnout in East and West Germany

In [ ]:
# Load data file
file = "data/statistic_id36658_wahlbeteiligung-bei-den-bundestagswahlen-nach-bundeslaendern-bis-2021.xlsx"
laender = pd.read_excel(file, sheet_name="Daten", skiprows=4, usecols="B:J")

# Rename column
laender = laender.rename(columns={"Unnamed: 1": "Bundesland"})

# Define a condition on which states belong to the east of Germany
ost = [
    "Berlin",
    "Brandenburg",
    "Mecklenburg-Vorpommern",
    "Sachsen",
    "Sachsen-Anhalt",
    "Thüringen",
]
cond = laender["Bundesland"].isin(ost)

# For each federal state, add belonging to east/west
laender.loc[cond, "Region"] = "Ostdeutschland"
laender.loc[~cond, "Region"] = "Westdeutschland"

# Add federal belonging for nationwide numbers
laender.loc[laender["Bundesland"] == "Bundesweit", "Region"] = "Bundesweit"

# Add categories for federal (A) and federal state (C)
laender.loc[laender["Bundesland"] == "Bundesweit", "Typ"] = "A"
laender.loc[laender["Bundesland"] != "Bundesweit", "Typ"] = "C"

# Change format from wide to long
laender = pd.melt(
    laender,
    id_vars=["Bundesland", "Region", "Typ"],
    var_name="Jahr",
    value_name="Wahlbeteiligung",
)

# Set year to datetype int
laender["Jahr"] = laender["Jahr"].astype("int64")

# Calculate mean for east/west and add it to the dataframe
west_ost = (
    laender[laender["Typ"] == "C"]
    .groupby(["Region", "Jahr"], as_index=False)["Wahlbeteiligung"]
    .mean()
)
west_ost["Bundesland"] = west_ost["Region"]

# Add category B for east/west averages
west_ost["Typ"] = "B"

# Add east/west means as rows to dataframe
laender = pd.concat([laender, west_ost])

In [ ]:
# Define colors to be used for the lines
colors = {
    "land_ost": "rgba(176, 11, 96, 0.1)",
    "region_ost": "rgba(176, 11, 96, 0.8)",
    "land_west": "rgba(1, 128, 0, 0.1)",
    "region_west": "rgba(1, 128, 0, 0.8)",
    "Bundesweit": "rgba(0, 0, 0, 0.8)",
}

# Create figure
fig = go.Figure()

# Filter dataframe for federal states
df_laender = laender[laender["Typ"] == "C"]

# Loop through federal states and create traces for each one
for land in df_laender["Bundesland"].unique():

    # Filter for current federal state
    trace_land = df_laender[df_laender["Bundesland"] == land]

    # Set the color to be used
    color = (
        "land_ost"
        if len(trace_land[trace_land["Region"] == "Ostdeutschland"]) > 0
        else "land_west"
    )

    # Add trace
    fig.add_traces(
        go.Scatter(
            x=trace_land["Jahr"],
            y=trace_land["Wahlbeteiligung"],
            mode="lines",
            name=land,
            line=dict(
                color=colors[color],
            ),
            showlegend=False,
        )
    )

# Filter dataframe for regions (east/west)
df_regionen = laender[laender["Typ"] == "B"]

# Loop through regions and create traces for each one
for region in df_regionen["Region"].unique():

    # Filter for current region
    trace_region = df_regionen[df_regionen["Region"] == region]

    # Set the color to be used
    color = (
        "region_ost"
        if len(trace_region[trace_region["Region"] == "Ostdeutschland"]) > 0
        else "region_west"
    )

    # Add trace
    fig.add_trace(
        go.Scatter(
            x=trace_region["Jahr"],
            y=trace_region["Wahlbeteiligung"],
            mode="lines",
            name=region,
            line=dict(
                color=colors[color],
                width=3,
            ),
        )
    )

# Filter dataframe for federal values
trace = laender[(laender["Bundesland"] == "Bundesweit") & (laender["Typ"] == "A")]

# Add trace
fig.add_trace(
    go.Scatter(
        x=trace["Jahr"],
        y=trace["Wahlbeteiligung"],
        mode="lines",
        name="Bundesweit",
        line=dict(
            color=colors["Bundesweit"],
            width=3,
        ),
    )
)

# Define basic settings
fig.update_layout(
    template=custom_template,
    margin=dict(r=30),
    title=dict(
        text=f"<b>Voter turnout ist especially low in East Germany</b><br />"
        f"<sup>At <span style='color:{colors['Bundesweit']}'><b>Federal level</b></span>, "
        f"in <span style='color:{colors['region_ost']}'><b>East Germany</b></span> and "
        f"in <span style='color:{colors['region_west']}'><b>West Germany</b></span></sup>",
    ),
    xaxis=dict(
        tickmode="array",
        tickvals=list(laender["Jahr"].unique()),
        title="",
        range=[laender["Jahr"].min() - 1, laender["Jahr"].max() + 1],
    ),
    yaxis=dict(
        range=[40, 90],
        title="Voter turnout (%)",
    ),
    showlegend=False,
)

# Add axis titles
fig = add_axis_titles(fig, "Election year", "Voter turnout (%)")

# Add attribution
fig = add_attribution(fig, "Bundeswahlleiter")

fig.show()

In [ ]:
# Export as image
fig.write_image("export/03-voter-turnout-east-west.png", scale=1)

## Get socio-economic data

In [ ]:
# Load socio-ecenomic data
structure_2021 = pd.read_csv(
    "data/btw2021_strukturdaten.csv", sep=";", decimal=",", thousands=".", skiprows=8
)

In [ ]:
# Rename columns
structure_2021 = structure_2021.rename(
    columns={
        "Wahlkreis-Nr.": "WahlkreisNr",
        "Land": "Bundesland",
        "Wahlkreis-Name": "Wahlkreis",
        "Gemeinden am 31.12.2019 (Anzahl)": "Gemeinden",
        "Fläche am 31.12.2019 (km²)": "Fläche",
        "Bevölkerung am 31.12.2019 - Insgesamt (in 1000)": "Bevoelkerung",
        "Bevölkerung am 31.12.2019 - Deutsche (in 1000)": "BevoelkerungDeutsche",
        "Bevölkerung am 31.12.2019 - Ausländer/-innen (%)": "BevoelkerungAusl",
        "Bevölkerungsdichte am 31.12.2019 (EW je km²)": "Bevoelkerungsdichte",
        "Zu- (+) bzw. Abnahme (-) der Bevölkerung 2019 - Geburtensaldo (je 1000 EW)": "BevGeburtensaldo",
        "Zu- (+) bzw. Abnahme (-) der Bevölkerung 2019 - Wanderungssaldo (je 1000 EW)": "BevWanderungssaldo",
        "Alter von ... bis ... Jahren am 31.12.2019 - unter 18 (%)": "AlterU18",
        "Alter von ... bis ... Jahren am 31.12.2019 - 18-24 (%)": "AlterU25",
        "Alter von ... bis ... Jahren am 31.12.2019 - 25-34 (%)": "AlterU35",
        "Alter von ... bis ... Jahren am 31.12.2019 - 35-59 (%)": "AlterU60",
        "Alter von ... bis ... Jahren am 31.12.2019 - 60-74 (%)": "AlterU75",
        "Alter von ... bis ... Jahren am 31.12.2019 - 75 und mehr (%)": "AlterÜ75",
        "Bodenfläche nach Art der tatsächlichen Nutzung am 31.12.2019 - Siedlung und Verkehr (%)": "FlaecheSiedlungVerkehr",
        "Bodenfläche nach Art der tatsächlichen Nutzung am 31.12.2019 - Vegetation und Gewässer (%)": "FlaecheVegetation",
        "Fertiggestellte Wohnungen 2019 (je 1000 EW)": "WohnungenFertig",
        "Bestand an Wohnungen am 31.12.2019 - insgesamt (je 1000 EW)": "WohnungenBestand",
        "Wohnfläche am 31.12.2019 (je Wohnung)": "Wohnflaeche",
        "Wohnfläche am 31.12.2019 (je EW)": "WohnflaecheEW",
        "PKW-Bestand am 01.01.2020 - PKW insgesamt (je 1000 EW)": "PKWs",
        "PKW-Bestand am 01.01.2020 - PKW mit Elektro- oder Hybrid-Antrieb (%)": "PKWElektro",
        "Unternehmensregister 2018 - Unternehmen insgesamt (je 1000 EW)": "Unternehmen",
        "Unternehmensregister 2018 - Handwerksunternehmen (je 1000 EW)": "UnternehmenHandwerk",
        "Schulabgänger/-innen beruflicher Schulen 2019": "SchulabgaengerBeruf",
        "Schulabgänger/-innen allgemeinbildender Schulen 2019 - insgesamt ohne Externe (je 1000 EW)": "SchulabgaengerAllg",
        "Schulabgänger/-innen allgemeinbildender Schulen 2019 - ohne Hauptschulabschluss (%)": "SchulabgaengerAllgOhneHaupt",
        "Schulabgänger/-innen allgemeinbildender Schulen 2019 - mit Hauptschulabschluss (%)": "SchulabgaengerAllgMitHaupt",
        "Schulabgänger/-innen allgemeinbildender Schulen 2019 - mit mittlerem Schulabschluss (%)": "SchulabgaengerAllgMittl",
        "Schulabgänger/-innen allgemeinblldender Schulen 2019 - mit allgemeiner und Fachhochschulreife (%)": "SchulabgaengerAllgHochsch",
        "Kindertagesbetreuung am 01.03.2020 - Betreute Kinder unter 3 Jahre (Betreuungsquote)": "KitaBetreuungsquoteU3",
        "Kindertagesbetreuung am 01.03.2020 - Betreute Kinder 3 bis unter 6 Jahre (Betreuungsquote)": "KitaBetreuungsquoteU6",
        "Verfügbares Einkommen der privaten Haushalte 2018 (EUR je EW)": "EinkommenEW",
        "Bruttoinlandsprodukt 2018 (EUR je EW)": "BIPperCapitaEW",
        "Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - insgesamt (je 1000 EW)": "BeschaeftigteSozPfl",
        "Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Land- und Forstwirtschaft, Fischerei (%)": "BeschaeftigteSozPflAgrar",
        "Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Produzierendes Gewerbe (%)": "BeschaeftigteSozPflProduzierend",
        "Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Handel, Gastgewerbe, Verkehr (%)": "BeschaeftigteSozPflHandelGastVerkehr",
        "Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Öffentliche und private Dienstleister (%)": "BeschaeftigteSozPflDienstl",
        'Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Übrige Dienstleister und "ohne Angabe" (%)': "BeschaeftigteSozPflDienstlAndere",
        "Empfänger/-innen von Leistungen nach SGB II  Oktober 2020 -  insgesamt (je 1000 EW)": "ALG2EW",
        "Empfänger/-innen von Leistungen nach SGB II  Oktober 2020 -  nicht erwerbsfähige Hilfebedürftige (%)": "ALG2NichtErwerbsfaehig",
        "Empfänger/-innen von Leistungen nach SGB II  Oktober 2020 -  Ausländer/-innen (%)": "ALG2Ausl",
        "Arbeitslosenquote Februar 2021 - insgesamt": "Erwerbslosenquote",
        "Arbeitslosenquote Februar 2021 - Männer": "ErwerbslosenquoteMaenner",
        "Arbeitslosenquote Februar 2021 - Frauen": "ErwerbslosenquoteFrauen",
        "Arbeitslosenquote Februar 2021 - 15 bis 24 Jahre": "ErwerbslosenquoteU25",
        "Arbeitslosenquote Februar 2021 - 55 bis 64 Jahre": "ErwerbslosenquoteU65",
    }
)

# Get data for constituencies (Wahlkreise)
# Because for some entities like Hamburg or Berlin there is no data
# at constituency level, we use the data for the whole city
filt1 = structure_2021["WahlkreisNr"] < 300  # All constituencies
filt2 = structure_2021["WahlkreisNr"].isin([902, 911])  # Hamburg, Berlin
filt = filt1 | filt2
structure_2021_const = structure_2021[filt].set_index("WahlkreisNr", drop=False)
structure_2021_const.index.name = None
structure_2021_const = structure_2021_const.sort_values("WahlkreisNr")

# Remove constituencies
drop = [*range(18, 24), *range(75, 87)]  # Hamburg (18-23)  # Berlin (75-86)
structure_2021_const = structure_2021_const.drop(labels=drop, axis=0)

# Change value for some constituencies
structure_2021_const.loc[902, "Wahlkreis"] = "Hamburg (total)"
structure_2021_const.loc[911, "Wahlkreis"] = "Berlin (total)"

# structure_2021_const

In [ ]:
# Change some values to percent (given in per 1.000)
structure_2021_const["ALG2EW"] = structure_2021_const["ALG2EW"] / 10

## Get election results

In [ ]:
# Get election results by constituency
results_2021 = pd.read_csv(
    "data/kerg2.csv",
    sep=";",
    decimal=",",
    thousands=".",
    skiprows=9,
)

In [ ]:
# Get participation for federal states
filt1 = results_2021["Gebietsart"] == "Land"
filt2 = results_2021["Gruppenart"] == "System-Gruppe"
filt3 = results_2021["Gruppenname"] == "Wählende"
filt = filt1 & filt2 & filt3
participation_2021_state = results_2021[filt]

participation_2021_state = (
    participation_2021_state[
        ["Gebietsnummer", "Gebietsname", "Prozent", "VorpProzent", "DiffProzent"]
    ]
    .sort_values("Gebietsnummer")
    .set_index("Gebietsnummer")
)
# participation_2021_state

In [ ]:
# Get participation for constituencies (Wahlkreise)
# Because for some entities like Hamburg or Berlin there is no socioeconomic
# data at constituency level, we use the data for the whole city
filt1 = results_2021["Gebietsart"] == "Wahlkreis"
filt2 = results_2021["Gebietsnummer"].isin([2, 11])  # Hamburg (2), Berlin (11)
filt3 = results_2021["Gruppenart"] == "System-Gruppe"
filt4 = results_2021["Gruppenname"] == "Wählende"
filt = (filt1 | filt2) & filt3 & filt4
participation_2021_const = results_2021[filt]

# Change numbers for Hamburg and Berlin
participation_2021_const.loc[204, "Gebietsnummer"] = 902  # Hamburg
participation_2021_const.loc[460, "Gebietsnummer"] = 911  # Berlin
# participation_2021_const

In [ ]:
# Get relevant columns only
participation_2021_const = (
    participation_2021_const[
        [
            "Gebietsnummer",
            "Gebietsname",
            "UegGebietsnummer",
            "Prozent",
            "VorpProzent",
            "DiffProzent",
        ]
    ]
    .sort_values("Gebietsnummer")
    .set_index("Gebietsnummer")
)

# Replace IDs with state names
states = dict(participation_2021_state["Gebietsname"])
participation_2021_const = participation_2021_const.replace(
    {"UegGebietsnummer": states}
)

# Change names for some constituencies
participation_2021_const.loc[
    participation_2021_const["Gebietsname"] == "Hamburg", "UegGebietsnummer"
] = "Hamburg"
participation_2021_const.loc[
    participation_2021_const["Gebietsname"] == "Berlin", "UegGebietsnummer"
] = "Berlin"

# Drop constituencies that are no longer needed
participation_2021_const = participation_2021_const.drop(labels=drop, axis=0)

# Rename columns
participation_2021_const = participation_2021_const.rename(
    columns={
        "Gebietsname": "Wahlkreis",
        "UegGebietsnummer": "Bundesland",
        "Prozent": "Wahlbeteiligung2021",
        "VorpProzent": "Wahlbeteiligung2017",
    }
)

# participation_2021_const

In [ ]:
# Exclude some columns
include_cols = list(structure_2021_const.columns)
unwanted = {"Bundesland", "WahlkreisNr", "Wahlkreis", "Fußnoten"}
include_cols = [col for col in include_cols if col not in unwanted]

In [ ]:
# Merge/join data for 2021
df = pd.merge(
    participation_2021_const,
    structure_2021_const[include_cols],
    left_index=True,
    right_index=True,
)

## Explore the data

In [ ]:
# Look for correlations
corr = (
    # Use numerical columns only
    df.select_dtypes(include=np.number)
    # Calculate correlations
    .corr()
    # Rename column we are interested in
    .rename(columns={"Wahlbeteiligung2021": "Correlation"})
    # Sort by renamed column
    .sort_values("Correlation")
    # Drop rows we don't need
    .drop(
        labels=[
            "DiffProzent",
            "Wahlbeteiligung2017",
            "Wahlbeteiligung2021",
        ],
        axis=0,
    )
    # Use just the one column we need
    [["Correlation"]]
)

# Only use relevant correlations
corr = corr[(corr["Correlation"] < -0.2) | (corr["Correlation"] > 0.2)]

# Rename labels
corr = corr.rename(
    index={
        "ALG2Ausl": "UnempBenefitForeigners",
        "ALG2EW": "UnempBenefit",
        "ALG2NichtErwerbsfaehig": "UnempBenefitNotFitForWork",
        "AlterU18": "AgeUnder18",
        "AlterU25": "AgeUnder25",
        "AlterU60": "AgeUnder60",
        "AlterÜ75": "AgeOver75",
        "AlterU75": "AgeUnder75",
        "BeschaeftigteSozPflAgrar": "EmployedSocialInsAgrar",
        "BeschaeftigteSozPflDienstlAndere": "EmployedSocialInsServices",
        "BevGeburtensaldo": "BalanceBirths",
        "BIPperCapitaEW": "GDPperCapita",
        "EinkommenEW": "Income",
        "Erwerbslosenquote": "Unemployment",
        "ErwerbslosenquoteFrauen": "UnemploymentWomen",
        "ErwerbslosenquoteMaenner": "UnemploymentMen",
        "ErwerbslosenquoteU25": "UnemploymentU25",
        "ErwerbslosenquoteU65": "UnemploymentU65",
        "KitaBetreuungsquoteU3": "ChildcareRateU3",
        "KitaBetreuungsquoteU6": "ChildcareRateU6",
        "PKWElektro": "NumCarsElectro",
        "PKWs": "NumCars",
        "SchulabgaengerAllgOhneHaupt": "SchoolDropout",
        "Unternehmen": "NumCompanies",
        "Wohnflaeche": "LivingSpace",
        "WohnungenBestand": "Appartments",
        "WohnungenFertig": "AppartmentsBuilt",
    }
)

# Have a look at the dataframe
# corr

In [ ]:
# Visualize correlations
fig = go.Figure(
    go.Bar(
        x=corr["Correlation"],
        y=corr.index,
        orientation="h",
        marker=dict(
            color=corr["Correlation"],
            colorscale="Viridis",
        ),
    )
)

fig.update_layout(
    template=custom_template,
    title=dict(
        text="<b>Strongest correlations: Income & unemployment rate under 25</b><br />"
        "<sup>Correlation with 2021 federal election voter turnout in Germany</sup>",
    ),
    margin=dict(r=30),
)

# Add axis titles
fig = add_axis_titles(fig, "Strength of Correlation")

# Add attribution
fig = add_attribution(fig, "Bundeswahlleiter", x=-0.2)

fig.show()

In [ ]:
# Export as image
fig.write_image("export/04-correlations.png", scale=1)

## Visualize different correlations

### Unemployment rate U25 & voter turnout

In [ ]:
# Visualise the relationship using Plotly Graph Objects (GO)
fig = go.Figure(
    go.Scatter(
        x=df["ErwerbslosenquoteU25"],
        y=df["Wahlbeteiligung2021"],
        mode="markers",
        marker=dict(
            color=df["DiffProzent"],
            colorscale="RdBu",
            size=abs(df["DiffProzent"]),
            sizeref=0.2,
            sizemin=2,
            colorbar=dict(
                outlinewidth=0,
                thickness=10,
                title=dict(text=""),
            ),
        ),
        # Add data to be used in hovertemplate
        customdata=df[
            ["Wahlkreis", "Bundesland", "ErwerbslosenquoteU25", "Wahlbeteiligung2021"]
        ],
        # Data to be displayed on hover
        hovertemplate="<br>".join(
            [
                "Constituency: <b>%{customdata[0]} (%{customdata[1]})</b>",
                "Unemployment rate: %{customdata[2]}",
                "Voter turnout 2021: %{customdata[3]}",
                "<extra></extra>",  # Remove secondary information
            ]
        ),
    )
)

fig.update_layout(
    title="<b>Voter turnout declines with rising unemployment</b><br />"
    "<sup>Unemployment of under 25-year-olds & voter turnout in the 2021 federal elections in Germany</sup>",
    template=custom_template,
    showlegend=False,
)

# Add axis titles
fig = add_axis_titles(fig, "Unemployment rate U25 (%)", "Voter turnout 2021 (%)")

# Add attribution
fig = add_attribution(fig, "Bundeswahlleiter")

# Add trendline
line = dict(color="#333", width=2)
fig = add_trendline(fig, line=line)

# Add colorbar title
add_colorbar_title(fig)

fig.show()

In [ ]:
# Export as image
fig.write_image("export/05-unemployment-voter-turnout.png", scale=1)

## Income & voter turnout

In [ ]:
# Visualise the relationship using Plotly Graph Objects (GO)
fig = go.Figure(
    go.Scatter(
        x=df["EinkommenEW"],
        y=df["Wahlbeteiligung2021"],
        mode="markers",
        marker=dict(
            color=df["DiffProzent"],
            colorscale="RdBu",
            size=abs(df["DiffProzent"]),
            sizeref=0.2,
            sizemin=2,
            colorbar=dict(
                outlinewidth=0,
                thickness=10,
                title=dict(text=""),
            ),
        ),
        # Add data to be used in hovertemplate
        customdata=df[
            ["Wahlkreis", "Bundesland", "EinkommenEW", "Wahlbeteiligung2021"]
        ],
        # Data to be displayed on hover
        hovertemplate="<br>".join(
            [
                "Constituency: <b>%{customdata[0]} (%{customdata[1]})</b>",
                "Income: %{customdata[2]}",
                "Voter turnout 2021: %{customdata[3]}",
                "<extra></extra>",  # Remove secondary information
            ]
        ),
    )
)

fig.update_layout(
    title="<b>The higher the income, the higher the voter turnout</b><br />"
    "<sup>Income & voter turnout in the 2021 federal elections in Germany</sup>",
    template=custom_template,
    showlegend=False,
)

# Add axis titles
fig = add_axis_titles(fig, "Income", "Voter turnout 2021 (%)")

# Add attribution
fig = add_attribution(fig, "Bundeswahlleiter")

# Add trendline
line = dict(color="#333", width=2)
fig = add_trendline(fig, line=line)

# Add colorbar title
add_colorbar_title(fig)

fig.show()

In [ ]:
# Export as image
fig.write_image("export/06-income-voter-turnout.png", scale=1)

## Subplots

Show some of these trends side by side.

In [ ]:
# Build Plotly figure with subplots
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Unemployment rate U25",
        "Income",
        "Recipients of unemployments benefits (ALG II)",
        "Number of companies",
    ),
)

# Add trace for unemployment rate
fig.add_trace(
    go.Scatter(
        x=df["ErwerbslosenquoteU25"],
        y=df["Wahlbeteiligung2021"],
        mode="markers",
        marker=dict(
            color=df["DiffProzent"],
            colorscale="RdBu",
            size=abs(df["DiffProzent"]),
            sizeref=0.5,
            sizemin=2,
        ),
    ),
    row=1,
    col=1,
)

# Add trace for income
fig.add_trace(
    go.Scatter(
        x=df["EinkommenEW"],
        y=df["Wahlbeteiligung2021"],
        mode="markers",
        marker=dict(
            color=df["DiffProzent"],
            colorscale="RdBu",
            size=abs(df["DiffProzent"]),
            sizeref=0.5,
            sizemin=2,
        ),
    ),
    row=1,
    col=2,
)

# Add trace for welfare benefits
fig.add_trace(
    go.Scatter(
        x=df["ALG2EW"],
        y=df["Wahlbeteiligung2021"],
        mode="markers",
        marker=dict(
            color=df["DiffProzent"],
            colorscale="RdBu",
            size=abs(df["DiffProzent"]),
            sizeref=0.5,
            sizemin=2,
        ),
    ),
    row=2,
    col=1,
)

# Add trace for number of companies
fig.add_trace(
    go.Scatter(
        x=df["Unternehmen"],
        y=df["Wahlbeteiligung2021"],
        mode="markers",
        marker=dict(
            color=df["DiffProzent"],
            colorscale="RdBu",
            size=abs(df["DiffProzent"]),
            sizeref=0.5,
            sizemin=2,
        ),
    ),
    row=2,
    col=2,
)

# Add trendlines
add_trendline(fig, pos=0, row=1, col=1)
add_trendline(fig, pos=1, row=1, col=2)
add_trendline(fig, pos=2, row=2, col=1)
add_trendline(fig, pos=3, row=2, col=2)

fig.update_layout(
    title="<b>Strong negative and positive correlations</b><br />"
    "<sup>Most likely influencing factors of voter turnout in German federal elections 2021</sup>",
    template=custom_template,
    showlegend=False,
)

fig = add_attribution(fig, "Bundeswahlleiter", x=-0.05)

fig.show()

In [ ]:
# Export as image
fig.write_image("export/07-subplots.png", scale=1)

## Bivariate map

Create a bivariate map to visualize correlations in a geographic context. For this example, we use the code documented before on [GitHub](https://github.com/yotkadata/plotly-bivariate-choropleth) and [Kaggle](https://www.kaggle.com/code/yotkadata/bivariate-choropleth-map-using-plotly).

In [ ]:
# Get shape file and open it
# https://www.bundeswahlleiter.de/bundestagswahlen/2021/wahlkreiseinteilung/downloads.html
shp_file = "data/shp/Geometrie_Wahlkreise_20DBT_geo.shp"
geodf = gpd.read_file(shp_file)

In [ ]:
# Merge the geospatial dataset with the data from above
df_biv = pd.merge(
    geodf,
    df[["Wahlbeteiligung2021", "ErwerbslosenquoteU25", "EinkommenEW"]],
    left_index=True,
    right_index=True,
)
# df_biv

In [ ]:
# Define some functions for later use

"""
Function to set default variables
"""


def conf_defaults():
    # Define some variables for later use
    conf = {
        "plot_title": "Bivariate choropleth map using Ploty",  # Title text
        "plot_title_size": 20,  # Font size of the title
        "width": 1000,  # Width of the final map container
        "ratio": 0.8,  # Ratio of height to width
        "center_lat": 0,  # Latitude of the center of the map
        "center_lon": 0,  # Longitude of the center of the map
        "map_zoom": 3,  # Zoom factor of the map
        "hover_x_label": "Label x variable",  # Label to appear on hover
        "hover_y_label": "Label y variable",  # Label to appear on hover
        "borders_width": 0.5,  # Width of the geographic entity borders
        "borders_color": "#f8f8f8",  # Color of the geographic entity borders
        # Define settings for the legend
        "top": 1,  # Vertical position of the top right corner (0: bottom, 1: top)
        "right": 1,  # Horizontal position of the top right corner (0: left, 1: right)
        "box_w": 0.04,  # Width of each rectangle
        "box_h": 0.04,  # Height of each rectangle
        "line_color": "#f8f8f8",  # Color of the rectagles' borders
        "line_width": 0,  # Width of the rectagles' borders
        "legend_x_label": "Higher x value",  # x variable label for the legend
        "legend_y_label": "Higher y value",  # y variable label for the legend
        "legend_font_size": 9,  # Legend font size
        "legend_font_color": "#333",  # Legend font color
        "attribution_font_size": 11,  # Legend font size
        "attribution_font_color": "#333",  # Legend font color
    }

    # Calculate height
    conf["height"] = conf["width"] * conf["ratio"]

    return conf


"""
Function to recalculate values in case width is changed
"""


def recalc_vars(new_width, variables, conf=conf_defaults()):

    # Calculate the factor of the changed width
    factor = new_width / 1000

    # Apply factor to all variables that have been passed to th function
    for var in variables:
        if var == "map_zoom":
            # Calculate the zoom factor
            # Mapbox zoom is based on a log scale. map_zoom needs to be set to value ideal for our map at 1000px.
            # So factor = 2 ^ (zoom - map_zoom) and zoom = log(factor) / log(2) + map_zoom
            conf[var] = math.log(factor) / math.log(2) + conf[var]
        else:
            conf[var] = conf[var] * factor

    return conf


"""
Function to load GeoJSON file with geographical data of the entities
"""


def load_geojson(geojson_url, data_dir="data", local_file=False):

    # Make sure data_dir is a string
    data_dir = str(data_dir)

    # Set name for the file to be saved
    if not local_file:
        # Use original file name if none is specified
        url_parsed = urlparse(geojson_url)
        local_file = os.path.basename(url_parsed.path)

    geojson_file = data_dir + "/" + str(local_file)

    # Create folder for data if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # Download GeoJSON in case it doesn't exist
    if not os.path.exists(geojson_file):

        # Make http request for remote file data
        geojson_request = requests.get(geojson_url)

        # Save file to local copy
        with open(geojson_file, "wb") as file:
            file.write(geojson_request.content)

    # Load GeoJSON file
    geojson = json.load(open(geojson_file, "r"))

    # Return GeoJSON object
    return geojson


"""
Function that assigns a value (x) to one of three bins (0, 1, 2).
The break points for the bins can be defined by break_a and break_b.
"""


def set_interval_value(x, break_1, break_2):
    if x <= break_1:
        return 0
    elif break_1 < x <= break_2:
        return 1
    else:
        return 2


"""
Function that adds a column 'biv_bins' to the dataframe containing the 
position in the 9-color matrix for the bivariate colors
    
Arguments:
    df: Dataframe
    x: Name of the column containing values of the first variable
    y: Name of the column containing values of the second variable

"""


def prepare_df(df, x="x", y="y"):

    # Check if arguments match all requirements
    if df[x].shape[0] != df[y].shape[0]:
        raise ValueError(
            "ERROR: The list of x and y coordinates must have the same length."
        )

    # Calculate break points at percentiles 33 and 66
    x_breaks = np.percentile(df[x], [33, 66])
    y_breaks = np.percentile(df[y], [33, 66])

    # Assign values of both variables to one of three bins (0, 1, 2)
    x_bins = [
        set_interval_value(value_x, x_breaks[0], x_breaks[1]) for value_x in df[x]
    ]
    y_bins = [
        set_interval_value(value_y, y_breaks[0], y_breaks[1]) for value_y in df[y]
    ]

    # Calculate the position of each x/y value pair in the 9-color matrix of bivariate colors
    df["biv_bins"] = [
        int(value_x + 3 * value_y) for value_x, value_y in zip(x_bins, y_bins)
    ]

    return df


"""
Function to create a color square containig the 9 colors to be used as a legend
"""


def create_legend(fig, colors, conf=conf_defaults()):

    # Reverse the order of colors
    legend_colors = colors[:]
    legend_colors.reverse()

    # Calculate coordinates for all nine rectangles
    coord = []

    # Adapt height to ratio to get squares
    width = conf["box_w"]
    height = conf["box_h"] / conf["ratio"]

    # Start looping through rows and columns to calculate corners the squares
    for row in range(1, 4):
        for col in range(1, 4):
            coord.append(
                {
                    "x0": round(conf["right"] - (col - 1) * width, 4),
                    "y0": round(conf["top"] - (row - 1) * height, 4),
                    "x1": round(conf["right"] - col * width, 4),
                    "y1": round(conf["top"] - row * height, 4),
                }
            )

    # Create shapes (rectangles)
    for i, value in enumerate(coord):
        # Add rectangle
        fig.add_shape(
            go.layout.Shape(
                type="rect",
                fillcolor=legend_colors[i],
                line=dict(
                    color=conf["line_color"],
                    width=conf["line_width"],
                ),
                xref="paper",
                yref="paper",
                xanchor="right",
                yanchor="top",
                x0=coord[i]["x0"],
                y0=coord[i]["y0"],
                x1=coord[i]["x1"],
                y1=coord[i]["y1"],
            )
        )

        # Add text for first variable
        fig.add_annotation(
            xref="paper",
            yref="paper",
            xanchor="left",
            yanchor="top",
            x=coord[8]["x1"],
            y=coord[8]["y1"],
            showarrow=False,
            text=conf["legend_x_label"] + " 🠒",
            font=dict(
                color=conf["legend_font_color"],
                size=conf["legend_font_size"],
            ),
            borderpad=0,
        )

        # Add text for second variable
        fig.add_annotation(
            xref="paper",
            yref="paper",
            xanchor="right",
            yanchor="bottom",
            x=coord[8]["x1"],
            y=coord[8]["y1"],
            showarrow=False,
            text=conf["legend_y_label"] + " 🠒",
            font=dict(
                color=conf["legend_font_color"],
                size=conf["legend_font_size"],
            ),
            textangle=270,
            borderpad=0,
        )

    return fig


"""
Function to create the map

Arguments:
    df: The dataframe that contains all the necessary columns
    colors: List of 9 blended colors
    x: Name of the column that contains values of first variable (defaults to 'x')
    y: Name of the column that contains values of second variable (defaults to 'y')
    ids: Name of the column that contains ids that connect the data to the GeoJSON (defaults to 'id')
    name: Name of the column conatining the geographic entity to be displayed as a description (defaults to 'name')
"""


def create_bivariate_map(
    df, colors, geojson, x="x", y="y", ids="id", name="name", conf=conf_defaults()
):

    if len(colors) != 9:
        raise ValueError(
            "ERROR: The list of bivariate colors must have a length eaqual to 9."
        )

    # Recalculate values if width differs from default
    # if not conf['width'] == 1000:
    #    conf = recalc_vars(conf['width'], ['height', 'plot_title_size', 'legend_font_size', 'attribution_font_size', 'map_zoom'], conf)

    # Prepare the dataframe with the necessary information for our bivariate map
    df_plot = prepare_df(df, x, y)

    # Create the figure
    fig = go.Figure(
        go.Choroplethmapbox(
            geojson=geojson,
            locations=df_plot[ids],
            z=df_plot["biv_bins"],
            marker_line_width=0.5,
            colorscale=[
                [0 / 8, colors[0]],
                [1 / 8, colors[1]],
                [2 / 8, colors[2]],
                [3 / 8, colors[3]],
                [4 / 8, colors[4]],
                [5 / 8, colors[5]],
                [6 / 8, colors[6]],
                [7 / 8, colors[7]],
                [8 / 8, colors[8]],
            ],
            customdata=df_plot[
                [name, ids, x, y]
            ],  # Add data to be used in hovertemplate
            hovertemplate="<br>".join(
                [  # Data to be displayed on hover
                    "<b>%{customdata[0]}</b> (ID: %{customdata[1]})",
                    conf["hover_x_label"] + ": %{customdata[2]}",
                    conf["hover_y_label"] + ": %{customdata[3]}",
                    "<extra></extra>",  # Remove secondary information
                ]
            ),
        )
    )

    # Add some more details
    fig.update_layout(
        title=dict(
            text=conf["plot_title"],
            font=dict(
                size=conf["plot_title_size"],
            ),
        ),
        mapbox_style="white-bg",
        width=conf["width"],
        height=conf["height"],
        autosize=True,
        mapbox=dict(
            center=dict(
                lat=conf["center_lat"], lon=conf["center_lon"]
            ),  # Set map center
            zoom=conf["map_zoom"],  # Set zoom
        ),
        template=custom_template,
    )

    fig.update_traces(
        marker_line_width=conf[
            "borders_width"
        ],  # Width of the geographic entity borders
        marker_line_color=conf[
            "borders_color"
        ],  # Color of the geographic entity borders
        showscale=False,  # Hide the colorscale
    )

    # Add the legend
    fig = create_legend(fig, colors, conf)

    return fig

In [ ]:
# Define sets of 9 colors to be used
# Order: bottom-left, bottom-center, bottom-right, center-left, center-center, center-right, top-left, top-center, top-right
color_sets = {
    "pink-blue": [
        "#e8e8e8",
        "#ace4e4",
        "#5ac8c8",
        "#dfb0d6",
        "#a5add3",
        "#5698b9",
        "#be64ac",
        "#8c62aa",
        "#3b4994",
    ],
    "teal-red": [
        "#e8e8e8",
        "#e4acac",
        "#c85a5a",
        "#b0d5df",
        "#ad9ea5",
        "#985356",
        "#64acbe",
        "#627f8c",
        "#574249",
    ],
    "blue-organe": [
        "#fef1e4",
        "#fab186",
        "#f3742d",
        "#97d0e7",
        "#b0988c",
        "#ab5f37",
        "#18aee5",
        "#407b8f",
        "#5c473d",
    ],
    "teal-red-grey": [
        "#e8e8e8",
        "#e4acac",
        "#c85a5a",
        "#b0d5df",
        "#adadad",
        "#985356",
        "#64acbe",
        "#627f8c",
        "#575757",
    ],
}

In [ ]:
"""
Get data and write it to a dataframe containing
    id: Id of the geographic entity (needs to be the same as references in the geospatial data)
    x: Values of the first variable
    y: Values of the second variable
"""

# Get data for first variable (x)
df_new = df_biv.copy()
df_new = df_new[["WKR_NR", "WKR_NAME", "Wahlbeteiligung2021", "EinkommenEW"]]

# Rename columns
df_new.columns = ["id", "name", "x", "y"]

In [ ]:
df_geojson = df_biv[["WKR_NR", "WKR_NAME", "geometry"]].set_index("WKR_NR")
geojson = json.loads(df_geojson.to_json())

In [ ]:
# Load conf defaults
conf = conf_defaults()

# Override some variables
conf["plot_title"] = "<b>Voter turnout and Income in German Federal Elections 2021</b>"
conf["plot_title_size"] = 24  # Font size of the title
conf["hover_x_label"] = "Voter turnout"  # Label to appear on hover
conf["hover_y_label"] = "Income"  # Label to appear on hover
conf["width"] = 1000
conf["ratio"] = 1
conf["height"] = conf["ratio"] * conf["width"]
conf["center_lat"] = 51.2  # Latitude of the center of the map
conf["center_lon"] = 11  # Longitude of the center of the map
conf["map_zoom"] = 5.65  # Zoom factor of the map
conf["borders_width"] = 0  # Width of the geographic entity borders

# Define settings for the legend
conf["line_width"] = 0  # Width of the rectagles' borders
conf["legend_x_label"] = "Higher voter turnout"  # x variable label for the legend
conf["legend_y_label"] = "Higher income"  # y variable label for the legend
conf["legend_font_size"] = 11  # Legend font size
conf["attribution_font_size"] = 11

In [ ]:
# Create our bivariate map
fig = create_bivariate_map(df_new, color_sets["teal-red-grey"], geojson, conf=conf)

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.01,
    y=0,
    showarrow=False,
    text="<b>Data:</b> COVID19-European-Regional-Tracker/Eurostat, "
    "<b>Graph:</b> Jan Kühn (https://yotka.org)",
    font=dict(
        size=conf["attribution_font_size"],
        color=conf["attribution_font_color"],
    ),
)

fig.update_layout(
    # paper_bgcolor='#4E5D6C',
    # plot_bgcolor='#4E5D6C',
    margin=dict(t=50, r=10, b=10, l=10),
    title_x=0.01,
    title_y=0.984,
)

fig.show()

In [ ]:
width_new = 1000

# Export as image
fig.write_image("export/08-bivariate-map.png", scale=width_new / conf["width"])

In [ ]:
# Useful tool to inspect Plotly figures
# fig_full = fig.full_figure_for_development(warn=False)
# print(fig_full.layout)